In [3]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Couldn't find program: 'bash'


In [4]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
okt = Okt()
kkma = Kkma()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import re
from collections import Counter

import warnings
warnings.simplefilter("ignore")

from wordcloud import WordCloud
import matplotlib.pyplot as plt


from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

# 기초 불용어 처리
from nltk.corpus import stopwords

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 형태소 벡터를 생성하기 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer
# 형태소 벡터를 학습 벡터로 변환한다.
from sklearn.feature_extraction.text import TfidfTransformer

# 워드 클라우드를 위한 라이브러리
from collections import Counter
# import pytagcloud
from IPython.display import Image

# 저장
import pickle

In [6]:
# 한국어 감성 사전 KOSAC 
# https://github.com/mrlee23/KoreanSentimentAnalyzer
# https://codingstudyroom.tistory.com/entry/%EC%9B%B9%ED%88%B0-%EC%8A%B9%EA%B2%A9-%ED%99%95%EB%A5%A0-%EC%98%88%EC%B8%A1-%EC%8B%9C%EC%8A%A4%ED%85%9C-%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-kosac%EC%82%AC%EC%A0%84%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D-%EA%B4%91%EC%A3%BC%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%ED%95%99%EC%9B%90

In [7]:
# 파일 불러오기 xlsx
path = r"C:\Users\NT550009\Desktop\11st\Filtered_stopwords.xlsx"
df_filtered = pd.read_excel(path)

In [8]:
df_filtered

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ID,Date,Grade,Text,Option,Size,ProductNum,FilteredText
0,0,0,0,날까,2023.04.08,4,쿠폰써서 2만원정도에 6벌이면 정말 싸요. 싼만큼 마감이 안좋은 부분도 있네요. 어...,"White 6-pack, 6, Medium",비공개,3569992365,"['쿠폰', '써서', '정도', '벌이', '정말', '싸요', '싼만큼', '마..."
1,1,1,1,newr*****,2023.03.27,5,옷에 비침도 크게 없고 사이즈도 넉넉해서 좋습니다. 이번 여름까지 무난하게 잘 입을...,"XX-Large, 6, White 6-pack",비공개,3569992365,"['비침도', '크게', '없고', '사이즈', '넉넉해서', '좋습니다', '여름..."
2,2,2,2,soch*****,2023.03.31,4,잘 알려진 브랜드입니다.면이 얇고 빨면 줄기도 합니다.올 여름 편하게 입으려고 구매...,"X-Large, 6, Charcoal Heather 6-pack",비공개,3569992365,"['알려진', '브랜드', '입니다면이', '얇고', '줄기', '합니다', '여름..."
3,3,3,3,2045***************,2023.03.02,5,얇아서 비침이 있으며 꽤 부드러운 재질입니다.이너로 입기 괜찮을듯,"6, Large, White 6-pack",비공개,3569992365,"['얇아서', '비침이', '있으며', '부드러운', '재질', '입니다', '입기..."
4,4,4,4,하이하이저요,2023.03.21,3,저렴하게 막 입기 좋네요생각보다 정사이즈인거 같아요 라지입는데 미듐샀더니가슴둘레가 ...,"Medium, 6, Navy 6-pack",비공개,3569992365,"['저렴하게', '입기', '좋네요', '생각', '보다', '정사', '이즈', ..."
...,...,...,...,...,...,...,...,...,...,...,...
7162,7162,698,698,파란마음,2021.11.23,3,찐짜크네요...팔길이가 유난희크네...,0,0,3596863528,"['크네요', '길이', '유난희', '크네']"
7163,7163,699,699,loze**,2021.11.22,5,가격대비좋아요,0,0,3596863528,"['가격', '대비', '좋아요']"
7164,7164,700,700,dnfl********,2021.11.19,4,만족합니다,0,0,3596863528,['만족합니다']
7165,7165,701,701,dnfl********,2021.11.19,4,한국사이즈100미디엄딱맞네요,0,0,3596863528,"['한국', '사이즈', '미디엄', '맞네요']"


In [9]:
import os 
currentPath = os.getcwd()
print(currentPath)

C:\Users\NT550009\Desktop\11st


In [12]:
# 원본

# import json

# class KnuSL():

# 	def data_list(wordname):	
# 		with open('data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
# 			data = json.load(f)
# 		result = ['None','None']	
# 		for i in range(0, len(data)):
# 			if data[i]['word'] == wordname:
# 				result.pop()
# 				result.pop()
# 				result.append(data[i]['word_root'])
# 				result.append(data[i]['polarity'])	
		
# 		r_word = result[0]
# 		s_word = result[1]
							
# 		print('어근 : ' + r_word)
# 		print('극성 : ' + s_word)		
		
		
# 		return r_word, s_word

# if __name__ == "__main__":
	
# 	ksl = KnuSL
	
# 	print("\nKNU 한국어 감성사전입니다~ :)")
# 	print("사전에 단어가 없는 경우 결과가 None으로 나타납니다!!!")
# 	print("종료하시려면 #을 입력해주세요!!!")
# 	print("-2:매우 부정, -1:부정, 0:중립 or Unkwon, 1:긍정, 2:매우 긍정")
# 	print("\n")	

# 	while(True):
# 		wordname = input("word : ")
# 		wordname = wordname.strip(" ")		
# 		if wordname != "#":
# 			print(ksl.data_list(wordname))
# 			print("\n")	
				

# 		elif wordname == "#":
# 			print("\n이용해주셔서 감사합니다~ :)")
# 			break


KNU 한국어 감성사전입니다~ :)
사전에 단어가 없는 경우 결과가 None으로 나타납니다!!!
종료하시려면 #을 입력해주세요!!!
-2:매우 부정, -1:부정, 0:중립 or Unkwon, 1:긍정, 2:매우 긍정


word : 가격이 싸다
어근 : 가격 싸
극성 : 1
('가격 싸', '1')




KeyboardInterrupt: Interrupted by user

word : #


In [20]:
# 함수를 바꿔보자

# 함수 목적에 맞게 바꾸기
# 리뷰별 값 정리하기 -> 사이즈가 언급된 리뷰의 값 확인하기
# 다른 요소가 언급된 리뷰의 값 확인하기
# 전반적으로 분포 확인

# 클래스 빼고 작동

# SentiWord_info 리스트 내 단어 변경 및 추가
def data_list(wordname): # 함수 이름 변경
    with open('data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
        data = json.load(f)
    result = ['None','None']
    for i in range(0, len(data)):
        if data[i]['word'] == wordname:
            result.pop()
            result.pop()
            result.append(data[i]['word_root'])
            result.append(data[i]['polarity'])

    r_word = result[0]
    s_word = result[1]

    print('어근 : ' + r_word)
    print('극성 : ' + s_word)


    return r_word, s_word

if __name__ == "__main__":

    ksl = KnuSL

print("\nKNU 한국어 감성사전입니다~ :)")
print("사전에 단어가 없는 경우 결과가 None으로 나타납니다!!!")
print("종료하시려면 #을 입력해주세요!!!")
print("-2:매우 부정, -1:부정, 0:중립 or Unkwon, 1:긍정, 2:매우 긍정")
print("\n")	

while(True):
    wordname = input("word : ")
    wordname = wordname.strip(" ")
    if wordname != "#":
        print(ksl.data_list(wordname)) 
        print("\n")

              

    elif wordname == "#":
        print("\n이용해주셔서 감사합니다~ :)")
        break


KNU 한국어 감성사전입니다~ :)
사전에 단어가 없는 경우 결과가 None으로 나타납니다!!!
종료하시려면 #을 입력해주세요!!!
-2:매우 부정, -1:부정, 0:중립 or Unkwon, 1:긍정, 2:매우 긍정


word : 가격이 싸다
어근 : 가격 싸
극성 : 1
('가격 싸', '1')


word : #

이용해주셔서 감사합니다~ :)


In [ ]:
# KNU 한국어 감성사전

# 형용사, 부사 단어 추가 : 좋다, / 나쁘다, 불만족
# 사전에 없는 단어

# polar = s_word 극값을 저장해서 리스트의 합계를 행에 저장

# FilteredText 행을 순회하면서 행 > 리스트 > 원소 
# 해당 리스트의 값 


# sample text = ['역시', '사이즈', '아주', '좋아요', '가격', '대비', '질도', '나쁘지', '않습니다']



In [ ]:
# sample text = ['역시', '사이즈', '아주', '좋아요', '가격', '대비', '질도', '나쁘지', '않습니다']

In [ ]:
# 함수 목적에 맞게 바꾸기
# 리뷰별 값 정리하기 -> 사이즈가 언급된 리뷰의 값 확인하기
# 다른 요소가 언급된 리뷰의 값 확인하기
# 전반적으로 분포 확인